<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#K-Core-Pruning" data-toc-modified-id="K-Core-Pruning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>K-Core Pruning</a></span></li><li><span><a href="#Spliting-Dataset" data-toc-modified-id="Spliting-Dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Spliting Dataset</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Regularization-적용하기" data-toc-modified-id="Regularization-적용하기-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Regularization 적용하기</a></span></li></ul></li><li><span><a href="#학습-데이터-구성하기" data-toc-modified-id="학습-데이터-구성하기-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>학습 데이터 구성하기</a></span><ul class="toc-item"><li><span><a href="#학습-데이터-구성하기" data-toc-modified-id="학습-데이터-구성하기-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>학습 데이터 구성하기</a></span></li><li><span><a href="#모델-학습하기" data-toc-modified-id="모델-학습하기-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>모델 학습하기</a></span></li></ul></li></ul></li></ul></div>

In [155]:
import os
import random
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.model_selection import train_test_split
import copy
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.metrics import RootMeanSquaredError, BinaryCrossentropy, SparseCategoricalAccuracy
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu

tqdm.pandas()
np.set_printoptions(precision=3)
plt.style.use("dark_background")

C:\Users\82104\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [156]:
users = pd.read_csv("Datasets/MovieLens 100k/100k_users.csv")
movies = pd.read_csv("Datasets/MovieLens 100k/100k_movies.csv")
ratings = pd.read_csv("Datasets/MovieLens 100k/100k_ratings.csv")

# K-Core Pruning

In [157]:
thr = 5
len_prev = -1
len_next = -2
while len_prev != len_next:
    len_prev = len(ratings)
    print(f"len(ratings): {len(ratings):,}")
    
    user_n_ratings = ratings["user_id"].value_counts()
    users_ = user_n_ratings[user_n_ratings>thr].index
    
    item_n_ratings = ratings["item_id"].value_counts()
    items_ = item_n_ratings[item_n_ratings>thr].index

    ratings = ratings[(ratings["user_id"].isin(users_)) & (ratings["item_id"].isin(items_))]
    len_next = len(ratings)
print("Finished!")

len(ratings): 99,991
len(ratings): 99,023
Finished!


# Spliting Dataset
- 시간 순서대로 Dataset을 나누겠습니다.

In [158]:
ratings_tr = pd.DataFrame()
ratings_te = pd.DataFrame()
for _, group in tqdm(ratings.groupby(["user_id"])):
    tr, te = train_test_split(group, test_size=0.1, shuffle=False)
    ratings_tr = pd.concat([ratings_tr, tr], axis=0)
    ratings_te = pd.concat([ratings_te, te], axis=0)

print(f"len(ratings_tr): {len(ratings_tr):,}")
print(f"len(ratings_te): {len(ratings_te):,}")


len(ratings_tr): 88,681
len(ratings_te): 10,342


In [159]:
movie_n_ratings = ratings.groupby(["item_id"]).size().sort_values(ascending=False)

X_te = copy.deepcopy(ratings_te)
y_te = ratings_te[["rating"]]
user_movies = ratings.groupby(["user_id"])["item_id"].apply(frozenset)
X_te["items"] = X_te["user_id"].apply(lambda x : user_movies[x])
X_te = X_te.drop(["rating"], axis=1)

# `item_id`: 본 영화 1개
# `items_100`: 보지 않은 영화 100개
X_te["items_100"] = X_te.progress_apply(lambda x:random.choices(list(x["items"] - {x["item_id"]}), k=100, weights=movie_n_ratings[list(x["items"] - {x["item_id"]})]), axis=1)

# def pick_items_100(x):
#     temp = movie_n_ratings[~movie_n_ratings.index.isin(x["items"])]
#     return set(temp.sample(100, replace=False, weights=movie_n_ratings).index)

# X_te["items_100"] = X_te.progress_apply(lambda x : pick_items_100(x), axis=1)

# Modeling

In [160]:
input_user = Input(shape=(), name="Input_user")
input_pos = Input(shape=(), name="Input_pos")
input_neg = Input(shape=(), name="Input_neg")
inputs = [input_user, input_pos, input_neg]

# n_users = ratings["user_id"].nunique()
# n_items = ratings["item_id"].nunique()
n_users = ratings["user_id"].max() + 1
n_items = ratings["item_id"].max() + 1
dim = 30
emb_user = Embedding(input_dim=n_users, output_dim=dim + 1, name="Embedding_user")
emb_item = Embedding(input_dim=n_items, output_dim=dim + 1, name="Embedding_item")

z1 = emb_user(input_user)
# z2 = emb_item(input_pos)
# z3 = emb_item(input_neg)
z2 = emb_item(input_pos)
z3 = emb_item(input_neg)

# 우리는 고객이 본 아이템에 대한 Score와 고객이 보지 않은 아이템에 대한 Score의 차이가 극대화되도록 학습하게 됩니다. 이를 위해 BPR에서는 Individual Probability, 즉 고객이 본 아이템에 대해 보지 않은 아이템보다 선호할 확률을 구하게 됩니다.
# Bayesian Personalized Ranking은 위의 확률이 100%가 되도록 학습합니다.
pos_score = Dot(axes=(1, 1))([z1, z2])
neg_score = Dot(axes=(1, 1))([z1, z3])
diff = pos_score - neg_score
outputs = sigmoid(diff)

# 입력값은 크게 세가지 input_user, input_pos,input_neg으로 나뉘어집니다. 그리고 출력값은 보지 않은 아이템에 대한 선호도보다 본 아이템에 대한 선호도가 높을 확률(individual probability)인 probs이 됩니다.
model = Model(inputs=inputs, outputs=outputs)

In [72]:
#유저 임베딩에서 우리는 마지막 임베딩에 1을 추가해주어야 합니다. 아이템 임베딩의 마지막 원소값 Bias를 추가하기 위함입니다.바로 아래와 같은 방식으로 유저 임베딩과 아이템 임베딩이 형성됩니다.
# Dot 연산으로 Bias 연산까지 같이 수행하기 위해 아래와 같이 코드를 작성하게 됩니다.

# z1 = emb_user(input_user)
# one_emb = tf.ones_like(user_emb[:, -1:])

U=[u1,u2,u3,...,u60,1]I=[i1,i2,i3,...,i60,ibias]

### Regularization 적용하기
- Matrix Factoriation은 쉽게 Overfitting, 즉 학습 데이터에만 과적합되는 현상이 발생합니다. 이를 방지하기 위해 가장 기본적인 방법론 중 하나는 Weight Decay, 즉 weight의 값이 너무 커지지 않도록 방지하는 것입니다. 이를 위해 아래와 같이 Loss를 추가해주게 되면, weight가 어느정도 줄어드는 방향으로 모델이 학습하게 됩니다.

In [161]:
# l2_user = z1**2
# l2_pos_item = z2**2
# l2_neg_item = z3**2
# l2_reg = 0.0001

# weight_decay = l2_reg*tf.reduce_sum(l2_user + l2_pos_item + l2_neg_item)

# model.add_loss(weight_decay)

model.compile(optimizer=Adagrad(1), loss="binary_crossentropy", metrics=["acc"])

## 학습 데이터 구성하기
- 우리가 가지고 있는 데이터는 고객이 특정 영화에 대해 몇점의 평점을 주었는지에 대한 데이터입니다. 우리는 고객이 평가하지 않은 영화에 대한 정보를 생성해서 Pair 단위로 모델을 학습해야 합니다.

### 학습 데이터 구성하기
우리는 매 Epoch마다 본것과 보지 않은 것에 대한 쌍을 무작위로 추출합니다. 그리고 학습 데이터에서 출력값은 항상 1로 나와야 합니다.(보지 않은 것에 대한 본것의 확률 = 100%)

In [162]:
all_movies = set(ratings_tr["item_id"])
user_not_movies = all_movies - user_movies
user_not_movies = user_not_movies.map(list)

def get_bpr_dataset(ratings_tr, user_not_movies):
    ratings_tr_batch = copy.deepcopy(ratings_tr)
#     ratings_tr_batch = ratings_tr_batch.sample(frac=1)
    ratings_tr_batch["neg_item"] = ratings_tr_batch.apply(lambda x : random.choice(user_not_movies[x["user_id"]]), axis=1)
    
#     x = {"Input_user":ratings_tr_batch["user_id"].values, "Input_pos":ratings_tr_batch["item_id"].values, "Input_neg":ratings_tr_batch["neg_item"].values}
    x = [ratings_tr_batch["user_id"].values, ratings_tr_batch["item_id"].values, ratings_tr_batch["neg_item"].values]
    y = np.ones(shape=(len(ratings_tr_batch), 1))
    
    return x, y

### 모델 학습하기
- epoch 10번에 걸쳐 모델을 학습시키도록 하겠습니다. 매 Epoch마다 새로운 학습 pair를 생성하도록 하였습니다.

In [163]:
# ratings_tr_batch = ratings_tr.copy()
# ratings_tr_batch = ratings_tr_batch.sample(frac=1)
# ratings_tr_batch["neg_item"] = ratings_tr_batch.apply(lambda x : random.choice(user_not_movies.loc[x["user_id"]]), axis=1)

In [164]:
n_epochs = 10
for i in range(1, n_epochs + 1):
    print(f"epoch: {i:>3d}")
    X, y = get_bpr_dataset(ratings_tr, user_not_movies)
    model.fit(x=X, y=y, batch_size=64, verbose=2)

epoch:   1
1386/1386 - 2s - loss: 0.6914 - acc: 0.5854 - 2s/epoch - 1ms/step
epoch:   2
1386/1386 - 1s - loss: 0.5640 - acc: 0.8013 - 1s/epoch - 898us/step
epoch:   3
1386/1386 - 1s - loss: 0.4198 - acc: 0.8178 - 1s/epoch - 916us/step
epoch:   4
1386/1386 - 1s - loss: 0.3671 - acc: 0.8447 - 1s/epoch - 914us/step
epoch:   5
1386/1386 - 1s - loss: 0.3198 - acc: 0.8682 - 1s/epoch - 891us/step
epoch:   6
1386/1386 - 1s - loss: 0.2810 - acc: 0.8846 - 1s/epoch - 936us/step
epoch:   7
1386/1386 - 1s - loss: 0.2573 - acc: 0.8948 - 1s/epoch - 916us/step
epoch:   8
1386/1386 - 1s - loss: 0.2380 - acc: 0.9032 - 1s/epoch - 976us/step
epoch:   9
1386/1386 - 1s - loss: 0.2170 - acc: 0.9117 - 1s/epoch - 997us/step
epoch:  10
1386/1386 - 1s - loss: 0.2079 - acc: 0.9159 - 1s/epoch - 958us/step


In [ ]:
model = Model(inputs=inputs, outputs=outputs)